In [3]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
import locale
from datetime import datetime
import statistics as stat

%matplotlib inline

In [4]:
dat = pd.read_csv('data/assistments17/preprocessed_data.csv', sep='\t')
print(len(dat['timestamp'].unique()))
dat.iloc[0]
target_dat = dat[(dat['user_id'] == 0) & (dat['skill_id'] == 206)]

665161


In [5]:
print(len(dat['skill_id'].unique()))
print(len(dat['user_id'].unique()))

411
1708


In [125]:
def calc_m(B_s, B_i, B_si, ts, ds, cur_t, types):
    ages = [cur_t - t + 1 for t in ts]
    tds = [a ** -ds[idx] for idx, a in enumerate(ages)]
#         tds = [a ** -.5 for idx, a in enumerate(ages)]
#     print(ages)
#     print(tds)
    if types:
        tds = np.multiply([1 if t else 0.5 for t in types], tds)
        
    m = B_s + B_i + B_si + np.log(np.sum(tds))
    return m
    

def calc_ms(B_s, B_i, B_si, c, alpha, ts, types=None):
    ms = []
    ds = []
    for k in range(len(ts)):
#         if k > 0:  print("{}, {}".format(np.exp(ms[k-1]), c * np.exp(ms[k-1])))
        dk = alpha if k == 0 else c * np.exp(ms[k-1]) + alpha
        ds.append(dk)
        m = calc_m(B_s, B_i, B_si, ts[:k+1], ds, ts[k], types[:k+1] if types else None)
        ms.append(m)
    return ms, ds

def calc_precall(T, s, m):
    return 1 / (1 + np.exp((T-m)/s))

def test(B_s, B_i, B_si, c, alpha, ts, ds, cur_t, types=None):
    m = calc_m(B_s, B_i, B_si, ts, ds, cur_t, types)
    return m

In [126]:
# Constants
B_s = 0
B_i = 0
B_si = 0
c = 0.15 # Decay scale parameter
alpha = 0.1 # Minimum rate of decay
T = 0 # Recall threshold
s = 0.26 # Noise in activation

In [127]:
def pp(ms, ds, ts):
    for m, d, t in zip(ms, ds, ts):
        print('m={:.5f}, d={:.5f}, precall={:.5f}'.format(m, d, calc_precall(T, s, m)))
#         print('{},{:.5f},{:.5f},{:.5f}'.format(t,m, d, calc_precall(T, s, m)))

def dtos(days):
    if isinstance(days, list): 
        return [d*86400 for d in days]
    return days * 86400

def mtos(minutes):
    if isinstance(minutes, list): 
        return [m*60 for m in minutes]
    return minutes * 60

def tos(day, hour, minute):
    return day*86400 + hour*3600 + minute*60
#     return day*1440 + hour*60 + minute

ts = [
    tos(0, 0, 0),
    tos(0, 0, 1),
    tos(0, 0, 2),
    tos(0, 0, 3),
    tos(0, 0, 4),
    tos(0, 0, 5),
    tos(0, 0, 6),
    tos(0, 0, 7),
    tos(0, 0, 8),
]
ms, ds = calc_ms(B_s, B_i, B_si, c, alpha, ts)
tm = test(B_s, B_i, B_si, c, alpha, ts, ds, tos(60, 0, 45))
print('==== massed ====')
pp(ms, ds, ts)
print('test: m={}, p_recall={}'.format(tm, calc_precall(T, s, tm)))

ts = [
    tos(0, 0, 0),
    tos(0, 12, 0),
    tos(1, 0, 0),
    tos(1, 12, 0),
    tos(2, 0, 0),
    tos(2, 12, 0),
    tos(3, 0, 0),
    tos(3, 12, 0),
    tos(4, 0, 0),
]

ms, ds = calc_ms(B_s, B_i, B_si, c, alpha, ts)
tm = test(B_s, B_i, B_si, c, alpha, ts, ds, tos(64, 0, 0))
print('==== spaced ====')
pp(ms, ds, ts)
print('test: m={}, p_recall={}'.format(tm, calc_precall(T, s, tm)))

==== massed ====
m=0.00000, d=0.10000, precall=0.50000
m=0.50858, d=0.25000, precall=0.87611
m=0.68151, d=0.34944, precall=0.93221
m=0.75794, d=0.39653, precall=0.94859
m=0.80358, d=0.42008, precall=0.95651
m=0.83667, d=0.43503, precall=0.96150
m=0.86290, d=0.44630, precall=0.96507
m=0.88461, d=0.45550, precall=0.96778
m=0.90307, d=0.46331, precall=0.96992
test: m=-1.402170853661944, p_recall=0.004527873042981066
==== spaced ====
m=0.00000, d=0.10000, precall=0.50000
m=0.29559, d=0.25000, precall=0.75710
m=0.32948, d=0.30159, precall=0.78027
m=0.34107, d=0.30854, precall=0.78782
m=0.35184, d=0.31097, precall=0.79466
m=0.36236, d=0.31325, precall=0.80118
m=0.37236, d=0.31551, precall=0.80724
m=0.38175, d=0.31767, precall=0.81280
m=0.39056, d=0.31973, precall=0.81790
test: m=-1.244838847954643, p_recall=0.008261594809946376


In [128]:
ts = [
    tos(0, 0, 0),
    tos(0, 0, 1),
    tos(0, 0, 2),
    tos(0, 0, 3),
    tos(0, 0, 4),
    tos(0, 0, 5),
    tos(0, 0, 6),
    tos(0, 0, 7),
    tos(0, 0, 8),
    tos(0, 0, 9),
]

types = [
    False, # True = test
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
]

ms, ds = calc_ms(B_s, B_i, B_si, c, alpha, ts, types)
tm = test(B_s, B_i, B_si, c, alpha, ts, ds, tos(1, 0, 45), types)
print('==== mixed ====')
pp(ms, ds, ts)
print('test: m={:.5f}, p_recall={:.10f}'.format(tm, calc_precall(T, s, tm)))

types = [
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
]
ms, ds = calc_ms(B_s, B_i, B_si, c, alpha, ts, types)
tm = test(B_s, B_i, B_si, c, alpha, ts, ds, tos(1, 0, 0), types)
print('==== study ====')
pp(ms, ds, ts)
print('test: m={:.5f}, p_recall={:.10f}'.format(tm, calc_precall(T, s, tm)))

==== mixed ====
m=-0.69315, d=0.10000, precall=0.06501
m=0.28628, d=0.17500, precall=0.75046
m=0.25972, d=0.29972, precall=0.73085
m=0.62870, d=0.29448, precall=0.91820
m=0.47520, d=0.38128, precall=0.86149
m=0.75079, d=0.34125, precall=0.94723
m=0.57942, d=0.41780, precall=0.90278
m=0.82092, d=0.36775, precall=0.95920
m=0.64614, d=0.44089, precall=0.92309
m=0.86908, d=0.38622, precall=0.96586
test: m=-0.89325, p_recall=0.0312020709
==== study ====
m=-0.69315, d=0.10000, precall=0.06501
m=-0.18457, d=0.17500, precall=0.32963
m=0.05168, d=0.22472, precall=0.54953
m=0.19213, d=0.25796, precall=0.67677
m=0.28786, d=0.28177, precall=0.75160
m=0.35880, d=0.30004, precall=0.79899
m=0.41430, d=0.31474, precall=0.83110
m=0.45936, d=0.32700, precall=0.85405
m=0.49696, d=0.33746, precall=0.87117
m=0.52899, d=0.34656, precall=0.88438
test: m=-0.97288, p_recall=0.0231611391


In [129]:
def day(x):
    return x * 86400.0
def hour(x):
    return x * 3600
def mins(x):
    return x * 60

massed_short_h = [
    day(12)+hour(7)+mins(50),
    day(12)+hour(7)+mins(51),
    day(12)+hour(7)+mins(52),
    day(12)+hour(7)+mins(53),
    day(12)+hour(7)+mins(54),
    day(12)+hour(7)+mins(55),
    day(12)+hour(7)+mins(56),
    day(12)+hour(7)+mins(57),
    day(12)+hour(7)+mins(58),
    day(12)+hour(7)+mins(59),
    day(12)+hour(8)+mins(0),
]

massed_long_h = [
    day(11)+hour(7)+mins(50),
    day(11)+hour(7)+mins(51),
    day(11)+hour(7)+mins(52),
    day(11)+hour(7)+mins(53),
    day(11)+hour(7)+mins(54),
    day(11)+hour(7)+mins(55),
    day(11)+hour(7)+mins(56),
    day(11)+hour(7)+mins(57),
    day(11)+hour(7)+mins(58),
    day(11)+hour(7)+mins(59),
    day(11)+hour(8)+mins(0),
]

spaced_short_h = [
    day(2)+hour(8)+mins(0),
    day(3)+hour(8)+mins(0),
    day(4)+hour(8)+mins(0),
    day(5)+hour(8)+mins(0),
    day(6)+hour(8)+mins(0),
    day(7)+hour(8)+mins(0),
    day(8)+hour(8)+mins(0),
    day(9)+hour(8)+mins(0),
    day(10)+hour(8)+mins(0),
    day(11)+hour(8)+mins(0),
    day(12)+hour(8)+mins(0),
]

spaced_long_h = [
    day(1)+hour(8)+mins(0),
    day(2)+hour(8)+mins(0),
    day(3)+hour(8)+mins(0),
    day(4)+hour(8)+mins(0),
    day(5)+hour(8)+mins(0),
    day(6)+hour(8)+mins(0),
    day(7)+hour(8)+mins(0),
    day(8)+hour(8)+mins(0),
    day(9)+hour(8)+mins(0),
    day(10)+hour(8)+mins(0),
    day(11)+hour(8)+mins(0),
]

now = day(12)+hour(8)+mins(10)

In [130]:
def run(B_s, B_i, B_si, c, alpha, ts, now):
    times = ts
    times.append(now)
    ms, ds = calc_ms(B_s, B_i, B_si, c, alpha, times)
    tm = test(B_s, B_i, B_si, c, alpha, ts, ds, now)
    pp(ms, ds, ts)
#     print('test: m={}, p_recall={}'.format(tm, calc_precall(T, s, tm)))
    
print('==== massed_short_h ====')
run(B_s, B_i, B_si, c, alpha, massed_short_h, now)

print('==== massed_long_h ====')
run(B_s, B_i, B_si, c, alpha, massed_long_h, now)

print('==== spaced_short_h ====')
run(B_s, B_i, B_si, c, alpha, spaced_short_h, now)

print('==== spaced_long_h ====')
run(B_s, B_i, B_si, c, alpha, spaced_long_h, now)

==== massed_short_h ====
m=0.00000, d=0.10000, precall=0.50000
m=0.50858, d=0.25000, precall=0.87611
m=0.68151, d=0.34944, precall=0.93221
m=0.75794, d=0.39653, precall=0.94859
m=0.80358, d=0.42008, precall=0.95651
m=0.83667, d=0.43503, precall=0.96150
m=0.86290, d=0.44630, precall=0.96507
m=0.88461, d=0.45550, precall=0.96778
m=0.90307, d=0.46331, precall=0.96992
m=0.91907, d=0.47008, precall=0.97166
m=0.93313, d=0.47604, precall=0.97312
m=0.77142, d=0.48137, precall=0.95106
==== massed_long_h ====
m=0.00000, d=0.10000, precall=0.50000
m=0.50858, d=0.25000, precall=0.87611
m=0.68151, d=0.34944, precall=0.93221
m=0.75794, d=0.39653, precall=0.94859
m=0.80358, d=0.42008, precall=0.95651
m=0.83667, d=0.43503, precall=0.96150
m=0.86290, d=0.44630, precall=0.96507
m=0.88461, d=0.45550, precall=0.96778
m=0.90307, d=0.46331, precall=0.96992
m=0.91907, d=0.47008, precall=0.97166
m=0.93313, d=0.47604, precall=0.97312
m=0.37157, d=0.48137, precall=0.80676
==== spaced_short_h ====
m=0.00000, d=0